In [2]:
%matplotlib qt

In [62]:
# process prediction and visualize

import numpy as np
#import matplotlib.pyplot as plt


pred = np.load('/Users/carotenuto/Documents/GitHub/3d-facial-landmarks-omfs/MeshCNN/datasets/headspace_cust1/predictions.pkl')
pred = np.reshape(pred, (68,3))
print(pred.shape)

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt



fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = pred[:,0]
y = pred[:,1]
z = pred[:,2]



ax.scatter(x, y, z, c='r', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

ValueError: Cannot load file containing pickled data when allow_pickle=False

In [ ]:
import open3d as o3d

def visualize(mesh):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(mesh)
    vis.run()
    vis.destroy_window()

def main():
    mesh = o3d.io.read_triangle_mesh("/Users/carotenuto/Documents/GitHub/3d-facial-landmarks-omfs/scripts/130926121017_reduced.obj")
    visualize(mesh)

main()

In [19]:
import plotly
from mpl_toolkits.mplot3d import Axes3D


In [20]:

import numpy as np
import meshio  
import plotly.graph_objects as go

In [21]:
with open('/Users/carotenuto/Documents/GitHub/3d-facial-landmarks-omfs/MeshCNN/datasets/headspace_cust1/00009/test/130926122455.obj','r') as file:
    obj_data = file.read()
print(obj_data)

####
#
# OBJ File Generated by Meshlab
#
####
# Object 130926122455.obj
#
# Vertices: 3994
# Faces: 8000
#
####
vn -2.368131 -3.797496 -3.644029
v -162.189239 -88.042678 -146.592215
vn -5.103770 -1.649043 -1.753519
v -165.368921 -88.891606 -141.578319
vn -1.271988 -3.950684 -4.160489
v -160.888919 -92.315357 -142.776651
vn 2.198215 -4.362197 -3.666721
v -155.511993 -89.535002 -144.366352
vn 1.566167 -4.280573 -4.137752
v -157.668891 -93.685532 -140.970049
vn -2.567310 -3.707934 -2.608694
v -169.917336 -130.293160 -117.079214
vn -5.159498 -0.771146 -1.242754
v -173.431310 -125.280634 -115.975195
vn -2.983365 -2.035747 1.750264
v -172.083005 -129.956660 -109.153985
vn 0.273055 -4.699331 0.110811
v -165.612223 -133.157066 -112.509776
vn 0.665595 -4.324502 -3.611115
v -165.587368 -130.707839 -117.506887
vn -2.054359 0.484476 5.160637
v -168.571191 -125.738111 -106.591739
vn -0.330822 -2.712778 4.127023
v -167.100772 -129.973455 -106.802893
vn 2.858747 -4.982404 -1.878123
v -160.989792 -130

In [22]:

def obj_data_to_mesh3d(odata):
    # odata is the string read from an obj file
    vertices = []
    faces = []
    lines = odata.splitlines()   
   
    for line in lines:
        slist = line.split()
        if slist:
            if slist[0] == 'v':
                vertex = np.array(slist[1:], dtype=float)
                vertices.append(vertex)
            elif slist[0] == 'f':
                face = []
                for k in range(1, len(slist)):
                    face.append([int(s) for s in slist[k].replace('//','/').split('/')])
                if len(face) > 3: # triangulate the n-polyonal face, n>3
                    faces.extend([[face[0][0]-1, face[k][0]-1, face[k+1][0]-1] for k in range(1, len(face)-1)])
                else:    
                    faces.append([face[j][0]-1 for j in range(len(face))])
            else: pass
    
    
    return np.array(vertices), np.array(faces) 

In [23]:
vertices, faces = obj_data_to_mesh3d(obj_data)


In [24]:

from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot
init_notebook_mode(connected=True)

In [25]:

vertices.shape

(3994, 3)

In [26]:

x, y, z = vertices[:,:3].T
I, J, K = faces.T

mesh = go.Mesh3d(
            x=-x,
            y=-y,
            z=z,
            vertexcolor=vertices[:, 3:], #the color codes must be triplets of floats  in [0,1]!!                      
            i=I,
            j=J,
            k=K,
            name='',
            showscale=False)

In [27]:

layout = go.Layout(title='Mesh3d from a Wavefront obj file',
                   font=dict(size=14, color='black'),
                   width=900,
                   height=800,
                   scene=dict(xaxis=dict(visible=False),
                              yaxis=dict(visible=False),  
                              zaxis=dict(visible=False), 
                              aspectratio=dict(x=1.5,
                                               y=0.9,
                                               z=0.5
                                         ),
                              camera=dict(eye=dict(x=1., y=1., z=0.5)),
                        ), 
                  paper_bgcolor='rgb(235,235,235)',
                  margin=dict(t=175)) 

In [28]:

pl_mygray= [[0.0, 'rgb(112, 123, 143)'],
           [0.25, 'rgb(133, 153, 165)'],
           [0.5,  'rgb(156, 184, 188)'],
           [0.75, 'rgb(185, 210, 210)'],
           [1.0,  'rgb(220, 233, 233)'],
           ]

In [29]:
mesh.update(intensity=z, colorscale=pl_mygray)
mesh.pop('vertexcolor')
layout.update(paper_bgcolor='rgb(50,50,50)', 
              font=dict(size=14, color='white'))
newfig = go.Figure(data=[mesh], layout=layout)
#iplot(newfig)

In [30]:
import pickle
#import torch
with open('/Users/carotenuto/Documents/GitHub/3d-facial-landmarks-omfs/MeshCNN/datasets/headspace_cust1/predictions.pkl', 'rb') as file:
    pred = pickle.load(file)
        
pred = np.reshape(pred, (68,3))
print(pred.shape)

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt



x = pred[:,0]
y = pred[:,1]
z = pred[:,2]

fig = go.Figure(data=[mesh], layout=layout)
fig.add_scatter3d(
    x=x,
    y=y,
z=z)

iplot(fig)

torch.Size([68, 3])


In [26]:

import chart_studio.plotly as py
py.iplot(newfig, filename='general-obj-2')

ModuleNotFoundError: No module named 'chart_studio'